In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyomo.environ import *

In [14]:
#Dados de Entrada
ger = ['Gerador_1', 'Gerador_2', 'Gerador_3', 'Gerador_4', 'Gerador_5']
Ng = len(ger)
pmin = [28,90,68,76,19]
pmax = [206,284,189,266,53]
c_ener_a = [3,4.05,4.05,3.99,3.88]
c_ener_b = [20,18.07,15.55,19.21,26.18]
c_ener_c = [100,98.87,104.26,107.21,95.31]

loads = ['Carga_1']
Nc = len(loads)
dem = [780]

In [15]:
# Criar Modelo Concreto
model = ConcreteModel('despacho_quad')

#Sets
model.ger = Set(initialize=ger)
model.loads = Set(initialize=loads)

#Parâmetros
model.pmin = Param(model.ger, initialize={ger[k]: pmin[k] for k in range(Ng)})
model.pmax = Param(model.ger, initialize={ger[k]: pmax[k] for k in range(Ng)})
model.c_ener_a = Param(model.ger, initialize={ger[k]: c_ener_a[k] for k in range(Ng)})
model.c_ener_b = Param(model.ger, initialize={ger[k]: c_ener_b[k] for k in range(Ng)})
model.c_ener_c = Param(model.ger, initialize={ger[k]: c_ener_c[k] for k in range(Ng)})

model.dem = Param(model.loads, initialize={loads[k]: dem[k] for k in range(Nc)})

#Variáveis
model.prod = Var(model.ger, domain = NonNegativeReals)

In [16]:
#Restrições
def prod_min(model, gerador):
    return model.prod[gerador] >= model.pmin[gerador]
model.prod_min = Constraint(model.ger, rule=prod_min)

def prod_max(model, gerador):
    return model.prod[gerador] <= model.pmax[gerador]
model.prod_max = Constraint(model.ger, rule=prod_max)

def balanco(model):
    return sum(model.prod[i] for i in model.ger) == sum(model.dem[j] for j in model.loads)
model.balanco = Constraint(rule=balanco)

In [17]:
#Função Objetivo:
def objective_rule(model):
    return sum(model.c_ener_a[gerador]*model.prod[gerador]*model.prod[gerador] + model.c_ener_b[gerador]*model.prod[gerador] + model.c_ener_c[gerador]  for gerador in model.ger)
model.objective = Objective(rule=objective_rule, sense=minimize)

#Executar Otimização:
solver = SolverFactory('ipopt')
resultados = solver.solve(model, tee=False)

In [18]:
# Relatório dos resultados de otimização
print('Status Final do Problema de Otimização:', resultados.solver.status, '\n')
print('Condição de Término:', resultados.solver.termination_condition, '\n')
print('Resultado Função Objetivo: R$', value(model.objective), '\n')

for gerador in model.ger:
    print('Energia gerada do {}: {:.2f} MWh'.format(gerador, model.prod[gerador].value))

Status Final do Problema de Otimização: ok 

Condição de Término: optimal 

Resultado Função Objetivo: R$ 518016.3337469104 

Energia gerada do Gerador_1: 206.00 MWh
Energia gerada do Gerador_2: 172.74 MWh
Energia gerada do Gerador_3: 173.06 MWh
Energia gerada do Gerador_4: 175.20 MWh
Energia gerada do Gerador_5: 53.00 MWh
